In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import scipy.stats as stats
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train_data = pd.read_csv('../input/data-storm-10/credit_card_default_train.csv')
test_data = pd.read_csv('../input/data-storm-10/credit_card_default_test.csv')

In [ ]:
def New_balance(x):
    if x[-1]=='M':
        x=float(x[:-1])*1000000
    elif x[-1]=='K':
        x=float(x[:-1])*1000
    else:
        x=float(x)
    return(x)

In [ ]:
train_data["Balance_Limit_V1_new"] = train_data["Balance_Limit_V1"].apply(New_balance)
train_data.drop("Balance_Limit_V1",axis=1,inplace=True)
train_data.rename(columns={"Balance_Limit_V1_new":"Balance_Limit_V1"},inplace=True)
train_col = list(train_data.columns)
last = train_col.pop()
train_col.insert(1,last)
train_data= train_data[train_col]

test_data["Balance_Limit_V1_new"] = test_data["Balance_Limit_V1"].apply(New_balance)
test_data.drop("Balance_Limit_V1",axis=1,inplace=True)
test_data.rename(columns={"Balance_Limit_V1_new":"Balance_Limit_V1"},inplace=True)
test_col = list(test_data.columns)
last = test_col.pop()
test_col.insert(1,last)
test_data= test_data[test_col]

In [ ]:
index = train_data.pop("Client_ID")
y = train_data.pop("NEXT_MONTH_DEFAULT")

index_test = test_data.pop("Client_ID")

In [ ]:
train_data["MARITAL_STATUS"].replace({"Other": "Other1"}, inplace=True)
test_data["MARITAL_STATUS"].replace({"Other": "Other1"}, inplace=True)

In [ ]:
changes=["Gender","AGE","EDUCATION_STATUS","MARITAL_STATUS"]
dummies=pd.concat([pd.get_dummies(train_data[col]) for col in changes], axis=1)
train_data=pd.concat([train_data,dummies],axis=1)

[train_data.pop(col) for col in changes]
train_data.head()

In [ ]:
dummies_test=pd.concat([pd.get_dummies(test_data[col]) for col in changes], axis=1)
test_data=pd.concat([test_data,dummies_test],axis=1)

[test_data.pop(col) for col in changes]
test_data.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( train_data, y, test_size=0.33 , random_state=7)

In [ ]:
X_train=X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values
test_data = test_data.values

In [ ]:
from sklearn.preprocessing import StandardScaler
scX = StandardScaler()
X_train1 = scX.fit_transform(np.concatenate((X_train[:,0:1],X_train[:,7:19]),axis=1))
X_test1 = scX.transform(np.concatenate((X_test[:,0:1],X_test[:,7:19]),axis=1))
test_data1 = scX.transform(np.concatenate((test_data[:,0:1],test_data[:,7:19]),axis=1))

In [ ]:
X_train1 = np.concatenate((X_train1[:,0:1],np.around(X_train1[:,1:],decimals=2, out=None),X_train[:,1:7],X_train[:,19:]),axis=1)
X_test1 = np.concatenate((X_test1[:,0:1],np.around(X_test1[:,1:],decimals=2, out=None),X_test[:,1:7],X_test[:,19:]),axis=1)
test_data1 = np.concatenate((test_data1[:,0:1],np.around(test_data1[:,1:],decimals=2, out=None),test_data[:,1:7],test_data[:,19:]),axis=1)

In [ ]:
import xgboost as xgb
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error,classification_report


In [ ]:
from xgboost import XGBClassifier
from matplotlib import pyplot
from xgboost import plot_importance

model = XGBClassifier()
model.fit(X_train1, y_train)

plot_importance(model)
pyplot.show()

In [ ]:

from numpy import sort
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel

thresholds = [0.0109]
for thresh in thresholds:
    # select features using threshold
    selection = SelectFromModel(model, threshold=thresh, prefit=True)
    select_X_train = selection.transform(X_train1)
    # train model
    selection_model = XGBClassifier(learning_rate =0.001,n_estimators=1000, max_depth=6, min_child_weight=1, gamma=0, subsample=0.8,
                                    colsample_bytree=0.8, objective= 'binary:logistic', nthread=4, scale_pos_weight=1,
                                    seed=30, random_state=7)

    selection_model.fit(select_X_train, y_train)
    # eval model
    select_X_test = selection.transform(X_test1)
    y_pred = selection_model.predict(select_X_test)
    predictions = [round(value) for value in y_pred]
    accuracy = accuracy_score(y_test, predictions)
    print("Thresh=%.4f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))


In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
select_test = selection.transform(test_data1)
prediction = selection_model.predict(select_test)

In [ ]:
#op = {'Client_ID':index_test,'NEXT_MONTH_DEFAULT':prediction}
#df = pd.DataFrame(op,columns=['Client_ID','NEXT_MONTH_DEFAULT'])
#vexport_csv=df.to_csv('prediction_day2_4.csv',index=None,header=True)